<a href="https://colab.research.google.com/github/donia-tl/chatbot/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


# NLP

In [ ]:
import nltk
nltk.download('punkt')  #pre trained tokenizer
import numpy as np

from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
  return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words): 
  #look at the tokenized sentence and return a bag of words (list of 1 or 0) if the words are in the senetence
  #at the respective positions
  #input : tokenized sentence
  #compare to all_words
  #output:bag of words
  
  tokenized_sentence = [stem(w) for w in tokenized_sentence]
  bag = np.zeros(len(all_words),dtype=np.float32)
  for idx,w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx]=1.0
  return bag





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Create training data

In [ ]:
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt

with open('intents.json','r') as f:
  intents =json.load(f)


all_words= []
tags=[]
xy=[]
for intent in intents['intents']:
  tag= intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w=tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words] #lower and not include punctuation then stem
all_words = sorted(set(all_words)) #convert to set to remove all duplicates and return as a list
tags = sorted(set(tags))


X_train=[] #will contain bag of words
y_train=[] #will contain the numbers for tags

for (pattern_sentence,tag) in xy:
  bag=bag_of_words(pattern_sentence,all_words)
  X_train.append(bag)
  label = tags.index(tag) #give the tags their labels/numbers
  y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset): #custom dataset
  def __init__(self):
    #data loading
    self.n_samples=len(X_train)
    self.x_data=X_train
    self.y_data=y_train

  def __getitem__(self,index):
    #helps index the dataset => dataset[0] = x,y
    return self.x_data[index], self.y_data[index]
  
  def __len__(self):
    #helps get len(dataset)
    return self.n_samples
  
#Hyperparameters
#devide data into batches for faster training
batch_size=8 #nb of training samples in one forward and backward pass
hidden_size=8
output_size=len(tags)
input_size=len(X_train[0]) #X = [nb_of_samples nb_of_features]
learning_rate=0.001
num_epochs=1000; # 1 epoch = one forward and backward pass of all training samples

#input size / batch size = nb of iterations per epoch

dataset=ChatDataset()
train_loader=DataLoader(dataset= dataset, batch_size=batch_size, shuffle=True, num_workers=2) #num_worker makes loading faster because it uses multiple subprocesses

device=torch.device('cpu')
model=NeuralNet(input_size,hidden_size,output_size).to(device) #design model by creating a custom one (class)

#loss and optimizer
criterion=nn.CrossEntropyLoss() #to measure the performance of the classification model (the lower the better)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate) #gradient descent 


for epoch in range(num_epochs): #training loop
  
  
  for (words,labels) in train_loader: #2nd loop to loop over the train loader
    words =words.to(device)
    labels=labels.to(device)

    #forward
    outputs= model(words) #outputs = prediction
    loss=criterion(outputs,labels) #y_pred and y

    #backward and optimizer step
    optimizer.zero_grad() # zero the gradient before every iteration
    loss.backward() #backward: gradient of the loss dl/dw
    optimizer.step() #update weights
    
  if (epoch +1) % 100 == 0: #print every 100 steps
    print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') #4 decimal values



print(f'final loss: {loss.item():.4f}')

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Epoch [100/1000], Loss: 0.0519
Epoch [200/1000], Loss: 0.0035
Epoch [300/1000], Loss: 0.0004
Epoch [400/1000], Loss: 0.0002
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0001
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000
training complete. file saved to data.pth


# Model


In [ ]:
import torch
import torch.nn as nn
class NeuralNet(nn.Module): #feedforward neural network with 2 hidden layers (input:bag of words)

  def __init__(self,input_size,hidden_size,num_classes): #num_classes = output size
    super(NeuralNet,self).__init__()
    #define layers and activation functions 
    self.l1=nn.Linear(input_size,hidden_size) #layer 1: linear layer wx=b: x input and b output
    self.l2=nn.Linear(hidden_size,hidden_size)
    self.l3=nn.Linear(hidden_size,num_classes)
    self.relu=nn.ReLU() #activation function

  def forward(self,x):
    out=self.l1(x)
    out=self.relu(out) #activation function after each layer for better learning
    out=self.l2(out)
    out=self.relu(out)
    out=self.l3(out)
    #no softmax function cuz its automatically applied by CrossEntropyLoss() 
    return out


# Chat

In [ ]:
import random
import json
import torch


device=torch.device('cpu')

with open('intents.json','r') as f:
  intents =json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model=NeuralNet(input_size,hidden_size,output_size).to(device)

model.load_state_dict(model_state)
model.eval()

bot_name = "DoniBot"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1) #torch.max(y_pred,1) to get the predictions

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1) #to get the probabilities of the predictions
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand... can you say that differently?")

Let's chat! (type 'quit' to exit)
You: hello
DoniBot: Good to see you again
You: how are you
DoniBot: i feel good and you?
You: im good
DoniBot: Good!
You: whats your name
DoniBot: You can call me DoniBot
You: nice to meet you donibot
DoniBot: Nice to meet you too :) 


KeyboardInterrupt: ignored